In [34]:
from pdfextractor import extractor
import pdfextractor
import pickle
import jieba
import pandas as pd
import re
import requests
import operator

In [288]:
class jmdPDFMap(object):
    """
    根据pdf生成文章，并计算推荐的准确率和召回率
    意图：导语，业绩变动原因，主营业务$业务进展，未来计划
    """
    def __init__(self):
        self.labels = ['导语', '业绩变动原因', '主营业务$业务进展', '未来计划', 'other']
        self.labelmap = {
                         '导语': ['意图_企业业绩_现状偏好', '意图_企业业绩_现状偏坏', '导语', '意图_企业基本信息介绍_企业基本信息介绍', '意图_企业活动_公司公告信息'],
                         '业绩变动原因': ['意图_企业业绩_业绩变动原因'],
                         '主营业务$业务进展': ['意图_企业业务_业务介绍', '意图_企业业务_业务规模', '意图_企业业务_业务规划', '意图_企业活动_人事活动', '意图_企业活动_融资活动','意图_企业活动_投资活动'],
                         '未来计划': ['意图_企业业务_业务前景', '意图_企业业绩_前景偏好','意图_企业业绩_前景偏好'],
                         'other': ['其他', '意图_评论_风险提示','意图_评论_投资建议', '意图_评论_盈利预测']
                        }
        self.labelmapR = {
                          '意图_企业业绩_现状偏好': '导语', '意图_企业业绩_现状偏坏': '导语', '导语': '导语', '意图_企业基本信息介绍_企业基本信息介绍': '导语', 
                          '意图_企业业绩_业绩变动原因': '业绩变动原因',
                          '意图_企业业务_业务介绍': '主营业务$业务进展', '意图_企业业务_业务规模': '主营业务$业务进展', '意图_企业业务_业务规划': '主营业务$业务进展', 
                          '意图_企业业务_业务前景': '未来计划', '意图_企业业绩_前景偏好': '未来计划', '意图_企业业绩_前景偏坏': '未来计划',
                          '意图_企业活动_公司公告信息': '主营业务$业务进展', '意图_企业活动_投资活动': '主营业务$业务进展', '意图_企业活动_人事活动': '主营业务$业务进展', '意图_企业活动_融资活动': '主营业务$业务进展', '其他': 'other', '意图_评论_风险提示': 'other', '意图_评论_投资建议': 'other', '意图_评论_盈利预测':'other'
                         }
        self.prop = [0.2, 0.2, 0.4, 0.3, 0]
        self.K = 0
        #每个意图对应的句子或段落数量
        self.num = {}
        self.recommendList = []
        
        
    def computePR(self, article, textList = None, pdfurl = None, recList = None, mode = 'para'):
        """
        pdfurl和textList为内容来源，必须有且仅有一个为None
        mode 必须指明
        """
        intentList = []
        #article_sen = self.get_article_sentence(article)
        article_sen = article
        article_len = len(article_sen)
        if article_len == 0:
            return -1, -1
        print 'article length: ' + str(article_len)
        if recList is None:
            if not pdfurl is None:
                if mode == 'para':
                    try:
                        pdftext = self.extractPDF(pdfurl, 0.8, ['para'])
                    except BaseException:
                        return -1, -1
                else:
                    try:
                        pdftext = self.extractPDF(pdfurl, 0.8, ['text'])
                    except BaseException:
                        return -1, -1
            else:
                pdftext = textList
            for t in pdftext:
                intentList.append(self.getIntent(t))
            if mode == 'para':
                self.K = int(len(article_sen) * 0.5) + 1
            else:
                self.K = int(len(article_sen) * 1.5) + 1
            for l in range(len(self.labels)):
                if l == len(self.labels) - 1:
                    self.num[self.labels[l]] = 0
                else:
                    self.num[self.labels[l]] = int(self.K * self.prop[l]) + 1
            self.recommendList = self.getTopKintent(intentList)
        else:
            self.recommendList = recList
        rec_len = len(self.recommendList)
        if rec_len == 0:
            return -1, -1
        print 'recommend list length: ' + str(rec_len)
        P_dic = {}
        Pcount = 0
        Rcount = 0
        for r in self.recommendList:
            P_dic[r] = False
        for sen in article_sen:
            for rec in self.recommendList:
                if self.MatchTwoText(sen, rec, 0.6):
                    Rcount += 1
                    if P_dic[rec] is False:
                        P_dic[rec] = True
                        Pcount += 1
                    break
        P = Pcount * 1.0 / rec_len
        R = Rcount * 1.0 / article_len
        print 'Pcount: ' + str(Pcount)
        print 'Rcount: ' + str(Rcount)
        return P, R
    
    def getRec(self):
        return self.recommendList
        
    def getTopKintent(self, intentlist):
        intent = {}
        for l in self.labels:
            intent[l] = {}
        
        for i in intentlist:
            key = i['intent']
            intent[key][i['content']] = i['prob']
        result = []
        for l in self.labels:
            intent_l = intent[l]
            intent[l] = sorted(intent_l.items(), key=operator.itemgetter(1))
        print "各个段落（句子）的预测意图分布为："
        for i in intent:
            print i + "  " + str(len(intent[i]))
        for l in intent.keys():
            if self.num[l] == 0:
                continue
            result.extend([d[0] for d in intent[l][-self.num[l]:]])
        return result
        
    def get_article_sentence(self, ar):
        """
        given an article, return the list of sentences
        """
        textList = ar.encode('utf-8').split('。') 
        textList = [t.replace('\n', '') for t in textList]
        textList = filter(lambda x: x != '', textList)
        return textList

    #percent: 取pdf的前多少页
    def extractPDF(self, pdfPath, percent, source = ['table', 'text', 'para']):
        savePath = '/home/zhuyuhe/mydata/jmd/percent/'
        ex = extractor.StockFinanceExtractor(pdfPath, percent)

        nodes = ex.get_pdf_objects(filter_headline=True)

        paraList = []
        textList = []
        textNodes = []
        tableList = []
        tableNodes = []
        senList = []
        for node in nodes:
            if isinstance(node.obj, pdfextractor.pdftree.VirtualNode):
                continue
            if isinstance(node.obj, pdfextractor.page.Table):
                tableNodes.append(node)
            elif  not isinstance(node.obj, pdfextractor.page.Catalog):
                textNodes.append(node)
        if 'table' in source:
            tableList = self.get_node_texts(tableNodes)
            tableList = [t.replace('|', '').split('\n') for t in tableList]
            #二维数组转化为一维
            tableList = sum(tableList,[])
        textList = self.get_node_texts(textNodes)
        if 'para' in source:
            paraList = [t.replace('\n', '') for t in textList]
            paraList = filter(lambda x: x != '', paraList)
        elif 'text' in source:
            senList = sum([t.encode('utf-8').split('。') for t in textList],[])
            senList = [t.replace('\n', '') for t in senList]
            senList = filter(lambda x: x != '', senList)

        pdf = []
    #     pdf.append(textList)
    #     pdf.append(tableList)
    #     pdf = sum(pdf, [])
#         print pdf
        pdf.extend(senList)
#         print 'after extent senList: ' + str(pdf) 
        pdf.extend(tableList)
#         print 'after extent tableList: ' + str(pdf) 
        pdf.extend(paraList)
#         print 'after extent paraList: ' + str(pdf) 
        print 'pdflist length: ' + str(len(pdf))

        return pdf
    
    def get_node_texts(self, pdf_nodes):
        '''
        Given a list of nodes, get the texts of these nodes
        '''
        texts = []
        for node in pdf_nodes:
            text = node.obj.get_text()
            if len(text) <= 25:
                continue
            if u'□' in text or u'√' in text:
                continue
            texts.append(text)
        return texts
    
    def getIntent(self, text):
        """
        return intent of the given text
        result: {'content': __, 'intent': __ , 'prob': __}
        """
        result = {}
        query = requests.post("https://surreal.aidigger.com/api/v1/du/finance",json=text).json()
        predict = query['predict']
        
        result['content'] = text
        result['intent'], result['prob'] = self.intentMap(predict)
        return result
    
    def intentMap(self, predict):
        """
        将现有意图映射到芥末堆所给的意图上去, 并将权重求和
        """
        result = {}
        for l in self.labels:
            result[l] = 0
#         for p in predict:
#             result[self.labelmapR[p['name'].encode('utf-8')]] += p['prob']
#         maxP = max(result.values())
#         intent = result.keys()[result.values().index(maxP)]
#         return intent, maxP
        for key in predict.keys():
            result[self.labelmapR[key.encode('utf-8')]] += predict[key]
        return result
        
        
    def MatchTwoText(self, t1, t2, threshold = 0.5):
        """
        检查两个句子是否匹配
        """
        ct1 = ' '.join(jieba.cut(t1)).split()
        ct2 = ' '.join(jieba.cut(t2)).split()
        commonL = set(ct1).intersection(set(ct2)).__len__()
        if commonL * 1.0 / set(ct1).__len__() > threshold:
            return True
        return False

    def splitByPeriod(self, P):
        """
        split paragraph to sentences
        """
        senList = sum([t.encode('utf-8').split('。') for t in P],[])
        senList = [t.replace('\n', '') for t in senList]
        senList = filter(lambda x: x != '', senList)
        return senList

In [340]:
def getPredict(mode = 'para'):
    for k in jmdata_map.keys():
        text = []
        for p in jmdata_map[k][mode]:
            text.append({'content': p})
        intent = requests.post("https://surreal.aidigger.com/api/v1/du/finance",json=text).json()
        intent = {x['content']: x['features']['intents'] for x in intent}
        for key, value in intent.items():
            intent[key] = {x['name']: x['prob'] for x in value}
            intent[key] = a.intentMap(intent[key])
        jmdata_map[k][mode+'Intent'] = intent
        
def getRec(mode = 'para'):
    num = {}
    labels = ['导语', '业绩变动原因', '主营业务$业务进展', '未来计划', 'other']
    prob = [0.2, 0.3,0.4,0.2,0]
    K = 0
    
    for pdfurl in jmdata_map.keys():
        if mode == 'para':
            K = 0.5 * len(jmdata_map[pdfurl]['content'])
        else:
            K = 1.5 * len(jmdata_map[pdfurl]['content'])
        for i in range(len(labels)):
            if i == 4:
                num[labels[i]] = 0
                continue
            num[labels[i]] = (int)(K * prob[i]) + 1
        recList = {}
        for i in labels[:-1]:
            recList[i] = []
        intent = {}
        for l in labels:
            intent[l] = {}
        for k, v in jmdata_map[pdfurl][mode+'Intent'].items():
            for i, p in v.items():
                intent[i][k] = p
        for i in intent.keys():
            intent[i] = sorted(intent[i].items(), key=operator.itemgetter(1))
            if num[i] == 0:
                continue
            recList[i] = [l[0] for l in intent[i][-num[i]:]]
        jmdata_map[pdfurl]['rec'+mode] = recList

jmdata_map = pickle.load(open('/home/zhuyuhe/mydata/jmd/percent/jmd_pdf_map.pkl', 'rb'))
url = []
for p in jmdata_map.keys():
    if jmdata_map[p]['mapcontent'].__len__() <= 5 and jmdata_map[p]:
        url.append(p)
for u in url:
    del jmdata_map[u]
#换了api，使用学优要求的推荐方法，先不确定每个句子的意图，每个意图推荐出前n个，然后取list(set())
getPredict('para')
getPredict('sen')

getRec('para')
getRec('sen')

In [341]:
#增加not_match_content_para, not_match_pdf_para,not_match_content_sen, not_match_pdf_sen
#not_match_content_para: 没有和推荐段落匹配上的文章句子
def MatchTwoText(t1, t2, pdflist,rmlist, threshold = 0.5):
        """
        检查两个句子是否匹配
        """
        ct1 = ' '.join(jieba.cut(t1)).split()
        ct2 = ' '.join(jieba.cut(t2)).split()
        commonL = set(ct1).intersection(set(ct2)).__len__()
        if commonL * 1.0 / set(ct1).__len__() > threshold:
            if not t2 in rmlist:
                rmlist.append(t2)
            return True
        return False

def getNotMatch(jmdata_map, key, mode):
    articleList = list(jmdata_map[key]['mapcontent'])
    rec = []
    for v in jmdata_map[key]['rec'+mode].values():
        rec.extend(v)
    rec = list(set(rec))
    print len(articleList), len(rec)
    rml = []
    rmr = []
    for ar in articleList:
        for text in rec:
            if MatchTwoText(ar, text, rec,rml, threshold=0.6):
                rmr.append(ar)
                break
    for r in rmr:
        articleList.remove(r)
    jmdata_map[key]['not_match_content_' + mode] = articleList
    not_match_pdf = {}
    modeIntent = jmdata_map[key][mode+'Intent']
    
    for r in rml:
        rec.remove(r)
    for r in rec:
        d = modeIntent[r]
        intent = d.keys()[d.values().index(max(d.values()))]
        not_match_pdf[r] = intent
    jmdata_map[key]['not_match_pdf_' + mode] = not_match_pdf
    
#jmdata_map = pickle.load(open('/home/zhuyuhe/mydata/jmd/percent/jmdata_map.pkl', 'rb'))
for key in jmdata_map.keys():
    getNotMatch(jmdata_map, key, 'para')
    getNotMatch(jmdata_map, key, 'sen')

9 9
9 21
12 11
12 29
7 8
7 21
9 10
9 23
7 8
7 18
6 9
6 21
9 7
9 22
8 9
8 21
9 11
9 26
15 11
15 30
13 10
13 30
7 13
7 31
10 8
10 21
22 16
22 48
8 11
8 21
8 11
8 28
9 9
9 22
6 8
6 20
6 8
6 19
9 8
9 21
6 6
6 15
16 15
16 37
20 15
20 43


In [345]:
jmdata_map[jmdata_map.keys()[0]]['recpara']

{'\xe4\xb8\x9a\xe7\xbb\xa9\xe5\x8f\x98\xe5\x8a\xa8\xe5\x8e\x9f\xe5\x9b\xa0': [u'5\u3001\u51c0\u5229\u6da6\u8f83 2015 \u5e74\u5ea6\u964d\u4f4e 90.62%\uff0c\u964d\u4f4e\u7684\u539f\u56e0\u4e00\u65b9\u9762\u56e0\u7ba1\u7406\u8d39\u7528\u589e\u957f\u5e45\u5ea6\u5927\u4e8e\u516c\u53f8\u9500\u552e\u6536\u5165\u589e\u957f\u901f\u5ea6\uff0c\u53e6\u4e00\u65b9\u9762\u516c\u53f8\u5411\u653f\u5e9c\u7533\u8bf7\u6302\u724c\u8865\u8d34\u5728 2016 \u5e74\u5c1a\u672a\u6536\u5230\uff0c\u8fd9\u6302\u724c\u8865\u8d34\u5f71\u54cd\u516c\u53f8\u5229\u6da6 100 \u4e07\u5143\uff0c\u5728\u62a5\u544a\u671f\u540e\uff0c\u516c\u53f8\u5df2\u6536\u5230\u653f\u5e9c\u5bf9\u6302\u724c\u4f01\u4e1a\u7684\u8865\u52a9 30 \u4e07\u3002',
  u'3\u3001\u8425\u4e1a\u5229\u6da6\u8f83 2015 \u5e74\u964d\u4f4e 129.18%\uff0c\u516c\u53f8\u7684\u8425\u4e1a\u6536\u5165\u867d\u4e0a\u6da8 11.35%\uff0c\u4f46\u662f\u4e0d\u53ca\u7ba1\u7406\u8d39\u7528\u4e2d\u5c24\u5176\u79d1\u6280\u7814\u53d1\u6295\u5165\u7684\u589e\u957f\u901f\u5ea6\uff0c\u7b

In [348]:
i = 0
import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

for key in jmdata_map.keys():
    fileObject = open('/home/zhuyuhe/mydata/jmd/rec/article' + str(i), 'w')
    fileObject.write('0. pdfurl:' + '\n')
    fileObject.write('---------------------' + '\n')
    fileObject.write(key)
    fileObject.write('\n\n\n' + '1. 文章内容为：' + '\n')
    fileObject.write('---------------------' + '\n')
    for s in jmdata_map[key]['content']:
        fileObject.write(s.encode('utf-8')+'\n')
        fileObject.write('-'+'\n')
    fileObject.write('\n\n\n' + '2. 来自pdf的文章内容为:'+'\n')
    fileObject.write('----------------------' + '\n')
    for s in jmdata_map[key]['mapcontent']:
        fileObject.write(s.encode('utf-8')+'\n')
        fileObject.write('-'+'\n')
    fileObject.write('\n\n\n' + '3. 推荐的pdf段落为:' + '\n')
    fileObject.write('-----------------------' + '\n')
    rp = jmdata_map[key]['recpara']
    for k in rp.keys():
        fileObject.write('---' + k + '--- '+ '\n')
        for r in rp[k]:
            fileObject.write(r + '\n')
            fileObject.write('-' + '\n')
        fileObject.write('---' + '\n')
    fileObject.write('\n\n\n' + '4. 没有和推荐段落匹配上的文章句子为：' + '\n')
    fileObject.write('------------------------' + '\n')
    for s in jmdata_map[key]['not_match_content_para']:
        fileObject.write(s.encode('utf-8') + '\n')
        fileObject.write('-' + '\n')
    
    fileObject.write('\n\n\n' + '5. 没有和文章句子匹配上的推荐段落及其意图为：' + '\n')
    fileObject.write('------------------------' + '\n')
    for k, v in jmdata_map[key]['not_match_pdf_para'].items():
        fileObject.write(k + '\t' + v + '\n')
        fileObject.write('-' + '\n')
    
    fileObject.write('\n\n\n' + '6. 推荐的pdf句子为:' + '\n')
    fileObject.write('-----------------------' + '\n')
    rs = jmdata_map[key]['recsen']
    for k in rs.keys():
        fileObject.write('---' + k + '--- '+ '\n')
        for r in rs[k]:
            fileObject.write(r + '\n')
            fileObject.write('-' + '\n')
        fileObject.write('---' + '\n')
        
    fileObject.write('\n\n\n' + '7. 没有和推荐句子匹配上的文章句子为：' + '\n')
    fileObject.write('------------------------' + '\n')
    for s in jmdata_map[key]['not_match_content_sen']:
        fileObject.write(s + '\n')
        fileObject.write('-' + '\n')
    
    
    fileObject.write('\n\n\n' + '没有和文章句子匹配上的推荐句子及其意图为：' + '\n')
    fileObject.write('------------------------' + '\n')
    for k,v in jmdata_map[key]['not_match_pdf_sen'].items():
        fileObject.write(k + '\t' + v + '\n')
        fileObject.write('-' + '\n')
    fileObject.close()
    i+=1

In [333]:
jmdata_map[jmdata_map.keys()[0]].keys()

['recPara2',
 'para',
 'mapcontent',
 'not_match_pdf_para',
 'senIntent',
 'recPara',
 'content',
 'not_match_content_para',
 'not_match_pdf_sen',
 'recSen',
 'paraIntent',
 'not_match_content_sen',
 'sen',
 'recpara',
 'recSen2',
 'recsen']

In [349]:
#content：文章内容,                     type:list
#mapcontent：来自pdf的文章内容          type:list
#para: 该pdf的段落列表                  type:list
#sen: 该pdf的句子列表                   type:list
#paraIntent：段落对应意图               type：{'句子'：{'意图'：prob}, '句子': {'意图':prob}}
#senIntent: 句子对应意图                type: 同paraIntent
#recpara: 推荐的pdf段落                 type: {'导语': ['句子', '句子'], '未来计划': ['句子', '句子']
#recsen: 推荐的pdf句子                  type：同recpara
#not_match_content_para: 没有和推荐段落匹配上的文章句子      type：list
#not_match_content_sen: 没有和推荐句子匹配上的文章句子       type: list
#not_match_pdf_para: 没有和文章句子匹配上的pdf段落及其意图   type: {'段落':'意图'}
#not_match_pdf_sen: 没有和文章句子匹配上的pdf句子及其意图    type: {'句子': '意图'}
with open('/home/zhuyuhe/mydata/jmd/percent/jmdata_map.pkl', 'wb') as pickle_file:
     pickle.dump(jmdata_map, pickle_file , protocol=2)

In [332]:
i = 1
print len(jmdata_map[jmdata_map.keys()[i]]['mapcontent'])

print len(jmdata_map[jmdata_map.keys()[i]]['recpara'])
print len(jmdata_map[jmdata_map.keys()[i]]['recsen'])
print len(jmdata_map[jmdata_map.keys()[i]]['not_match_content_para'])
print len(jmdata_map[jmdata_map.keys()[i]]['not_match_content_sen'])
print len(jmdata_map[jmdata_map.keys()[i]]['not_match_pdf_para'])
print len(jmdata_map[jmdata_map.keys()[i]]['not_match_pdf_sen'])

print len(jmdata_map[jmdata_map.keys()[i]]['para'])
print len(jmdata_map[jmdata_map.keys()[i]]['sen'])
print jmdata_map.keys()[i]


12
12
30
9
8
9
26
145
255
http://disclosure.szse.cn/finalpage/2017-04-21/1203344261.PDF


In [317]:
a = jmdPDFMap()
#jmdata_map = pickle.load(open('/home/zhuyuhe/mydata/jmd/percent/jmd_pdf_map.pkl', 'rb'))
P_para_map = []
R_para_map = []
P_sen_map = []
R_sen_map = []
for key in jmdata_map.keys():
    print key  
    print '---------PARA-----------'
    P,R = a.computePR(jmdata_map[key]['mapcontent'], recList=jmdata_map[key]['recpara'])
    P_para_map.append(P)
    R_para_map.append(R)
    print "Pecision: " + str(P) + "    Recall: " + str(R)
    print '---------SEN-------------'
    P,R = a.computePR(jmdata_map[key]['mapcontent'], recList=jmdata_map[key]['recsen'])
    P_sen_map.append(P)
    R_sen_map.append(R)
    print "Pecision: " + str(P) + "    Recall: " + str(R)
    print '*******************************************************'

http://www.cninfo.com.cn/finalpage/2017-04-19/1203340942.PDF
---------PARA-----------
article length: 9
recommend list length: 10
Pcount: 1
Rcount: 3
Pecision: 0.1    Recall: 0.333333333333
---------SEN-------------
article length: 9
recommend list length: 22
Pcount: 2
Rcount: 3
Pecision: 0.0909090909091    Recall: 0.333333333333
*******************************************************
http://disclosure.szse.cn/finalpage/2017-04-21/1203344261.PDF
---------PARA-----------
article length: 12
recommend list length: 12
Pcount: 3
Rcount: 3
Pecision: 0.25    Recall: 0.25
---------SEN-------------
article length: 12
recommend list length: 30
Pcount: 4
Rcount: 4
Pecision: 0.133333333333    Recall: 0.333333333333
*******************************************************
http://static.sse.com.cn/disclosure/listedinfo/announcement/c/2017-04-15/601801_2016_n.pdf
---------PARA-----------
article length: 7
recommend list length: 9
Pcount: 1
Rcount: 1
Pecision: 0.111111111111    Recall: 0.142857142857
-

In [316]:
#结果为-1代表mapcontent长度为0
a = jmdPDFMap()
#jmdata_map = pickle.load(open('/home/zhuyuhe/mydata/jmd/percent/jmd_pdf_map.pkl', 'rb'))
url = []
for p in jmdata_map.keys():
    if jmdata_map[p]['mapcontent'].__len__() <= 5 and jmdata_map[p]:
        url.append(p)
for u in url:
    del jmdata_map[u]
P_para_map = []
R_para_map = []
P_sen_map = []
R_sen_map = []
for key in jmdata_map.keys():
    print key  
    print '---------PARA-----------'
    P,R = a.computePR(jmdata_map[key]['mapcontent'], textList=jmdata_map[key]['para'], mode = 'para')
    P_para_map.append(P)
    R_para_map.append(R)
    jmdata_map[key]['recPara'] = a.getRec()
    print "Pecision: " + str(P) + "    Recall: " + str(R)
    print '---------SEN-------------'
    P,R = a.computePR(jmdata_map[key]['mapcontent'], textList=jmdata_map[key]['sen'], mode = 'sen')
    P_sen_map.append(P)
    R_sen_map.append(R)
    jmdata_map[key]['recSen'] = a.getRec()
    print "Pecision: " + str(P) + "    Recall: " + str(R)
    print '*******************************************************'
    

http://www.cninfo.com.cn/finalpage/2017-04-19/1203340942.PDF
---------PARA-----------
article length: 9


KeyError: 'predict'

In [329]:
text = {'content':'2003年前，任教于中国科学技术大学'}
query = requests.post("https://surreal.aidigger.com/api/v1/du/finance",json=text).json()
predict = query[0]['features']['intents']
# prob = [x['prob'] for x in predict]
# i = max(prob)
# intent = predict[prob.index(i)]['name']
# print intent
for i in predict:
    print i['name'], i['prob']

意图_企业业绩_现状偏好 0.0303134600859
意图_企业业绩_现状偏坏 0.0400226827081
意图_企业业务_业务前景 0.0467662658824
意图_企业业务_业务规划 0.0436516138708
意图_企业业务_业务规模 0.0493170299364
意图_企业业务_业务介绍 0.144790515798
意图_企业活动_公司公告信息 0.032699136721
意图_企业活动_投资活动 0.0396788936686
意图_企业业绩_前景偏好 0.0299068878646
意图_企业业绩_前景偏坏 0.00347500724781
意图_企业活动_人事活动 0.00678235790352
意图_企业活动_融资活动 0.00562315851967
意图_企业基本信息介绍_企业基本信息介绍 0.0107889492896
意图_企业业绩_业绩变动原因 0.339494465053
其他 0.0958497387547
意图_评论_风险提示 0.0466737285409
意图_评论_投资建议 0.0227112940676
意图_评论_盈利预测 0.00571671782108
导语 0.00573809626573


In [319]:
print P_para_map
print R_para_map
print P_sen_map
print R_sen_map

[0.1, 0.25, 0.1111111111111111, 0.2727272727272727, 0.2727272727272727, 0.2222222222222222, 0.3, 0.5, 0.2, 0.4166666666666667, 0.08333333333333333, 0.08333333333333333, 0.14285714285714285, 0.2222222222222222, 0.3333333333333333, 0.17647058823529413, 0.1, 0.3333333333333333, 0.1111111111111111, 0.1111111111111111, 0.42857142857142855, 0.1875, 0.375]
[0.3333333333333333, 0.25, 0.14285714285714285, 0.4444444444444444, 0.38461538461538464, 0.5714285714285714, 0.8333333333333334, 1.0, 0.375, 0.7777777777777778, 0.13333333333333333, 0.125, 0.2857142857142857, 0.4, 1.0, 0.18181818181818182, 0.2222222222222222, 0.8333333333333334, 0.16666666666666666, 0.1111111111111111, 0.5, 0.3125, 0.7]
[0.09090909090909091, 0.13333333333333333, 0.0, 0.25, 0.1935483870967742, 0.15789473684210525, 0.13636363636363635, 0.34782608695652173, 0.13636363636363635, 0.25925925925925924, 0.06451612903225806, 0.034482758620689655, 0.125, 0.18181818181818182, 0.18181818181818182, 0.10204081632653061, 0.173913043478260

In [318]:
import numpy as np
print 'p_para mean: ' + str(np.mean(filter(lambda x: x!=-1, P_para_map)))
print 'r_para mean: ' + str(np.mean(filter(lambda x: x!=-1, R_para_map)))
print 'p_sen mean: ' + str(np.mean(filter(lambda x: x!=-1, P_sen_map)))
print 'r_sen mean: ' + str(np.mean(filter(lambda x: x!=-1, R_sen_map)))

p_para mean: 0.231897020995
r_para mean: 0.438456048782
p_sen mean: 0.132069170595
r_sen mean: 0.385201935745


In [153]:
u = 'http://www.cninfo.com.cn/finalpage/2017-04-27/1203451818.PDF'
print u
for i in jmdata_map[u]['mapcontent']:
    print '-----------'
    print i
print '*************************************'

print 'recommend Paragraph:'
for i in jmdata_map[u]['recPara']:
    print '-----------'
    print i
print '***************************************'

print 'recommend Sentence:'
for i in jmdata_map[u]['recSen']:
    print '------------'
    print i

http://www.cninfo.com.cn/finalpage/2017-04-27/1203451818.PDF
-----------
 成立于 2006 年的世纪明德是专业的游学服务提供商，为 3 至 18 岁青少年提供国内游学、国际游学、研学旅行服务，业务涵盖游学产品研发、推广销售、团队运作和后续接待服务等全套流程
-----------
此外，其还面向行业提供教师培训和教育论坛服务
-----------
 报告显示，世纪明德在去年新增国内常规游学线路 14 条，国际游学线路 6 条，师训产品实现了体系化和模块化，客户学校增加了 47.38%
-----------
对于营收等经营指标的增长，报告给出了 3 点原因： 市场环境向好：国家研学旅行政策不断落地，政策环境向好，市场需求大增，客户人数增长迅速； 产品线丰富：加强了国内研学、国际游学、社会实践以及教师培训产品的研发，产品可以覆盖更大客户群，可以为存量客户提供更多的产品服务内容； 客户范围不断扩大：其去年对春秋季研学旅行市场进行了重点突破，从客户参营时间上进行延展，从一年中寒暑假的游学旺季拓展为全年旺季； 从各产品线的营收来看，国内研学产品仍然是世纪明德最重要的业务，营收占比达到 72.33%，收入增长 34.36%
-----------
 世纪明德方面表示，十余年来的积累使得其已经具备了规模优势和亿元级别的销售额，以及近三十万终端客户
-----------
未来，其将利用先发优势，利用投资并购手段，整合部分小型的游学机构
-----------
在业务层面，世纪明德将重点发力国际游学、营地的开发建设以及营地教育，通过设立教育产业并购的方式对行业进行整合
-----------
未来三到五年，世纪明德希望成为中国最大的综合性游学集团
-----------
 报告称，世纪明德在 2017 年计划完成 30 万人次的销售和接待量，实现营业收入持续快速增长
*************************************
recommend Paragraph:
-----------
公司的教师培训业务主要为教育工作者提供学习交流论坛，通过了解客户对于教育培训方面的需求，
-----------
产品研发方面，公司课程研发部门研发百余种研学课程，并汇集成册，形成了“研学旅行在中国”系

In [ ]:
jmdata_mapping = pickle.load(open('/home/zhuyuhe/mydata/jmd/percent/jmd_pdf_mapping2.pkl', 'rb'))[2:]
a = jmdPDFMap()
P_para = []
R_para = []
P_sen = []
R_sen = []
i = 0
for jmap in jmdata_mapping:
    print '******************************************************'
    print 'article ' + str(i) + ' :'
    print '--------------------'
    P,R = a.computePR(jmap['content'], jmap['pdf_url'], 'para')
    P_para.append(P)
    R_para.append(R)
    print "--PARA--  Pecision: " + str(P) + "    Recall: " + str(R)
    print '--------------------'
    P,R = a.computePR(jmap['content'], jmap['pdf_url'], 'sentence')
    P_sen.append(P)
    R_sen.append(R)
    print "--SEN--  Pecision: " + str(P) + "    Recall: " + str(R)
    print '---------------------'
    print '*******************************************************'
    i += 1
#print a.getIntent('前三季度利润为0，同比下降5%')['intent']

# jmap = jmd_mapping[0]
# print a.computePR(jmap['content'], jmap['pdf_url'], 'sentence')

In [ ]:
a = jmdPDFMap()
for jmap in jmdata_mapping[14:]:
    print '******************************************************'
    print 'article ' + str(i) + ' :'
    print '--------------------'
    P,R = a.computePR(jmap['content'], jmap['pdf_url'], 'para')
    P_para.append(P)
    R_para.append(R)
    print "--PARA--  Pecision: " + str(P) + "    Recall: " + str(R)
    print '--------------------'
    P,R = a.computePR(jmap['content'], jmap['pdf_url'], 'sentence')
    P_sen.append(P)
    R_sen.append(R)
    print "--SEN--  Pecision: " + str(P) + "    Recall: " + str(R)
    print '---------------------'
    print '*******************************************************'
    i += 1

In [ ]:
print 'P_para: ' + str(P_para)
print 'R_para: ' + str(R_para)
print 'P_sen: ' + str(P_sen)
print 'R_sen: ' + str(R_sen) 
print len(P_para)

In [ ]:
i = 0
jmdata_map = {}
P_para2 = []
P_sen2 = []
R_para2 = []
R_sen2 = []
jmdata_mapping = pickle.load(open('/home/zhuyuhe/mydata/jmd/percent/jmd_pdf_mapping2.pkl', 'rb'))[2:]
a = jmdPDFMap()
for jmap in jmdata_mapping:
    key = jmap['pdf_url']
    try:
        para = a.extractPDF(jmap['pdf_url'], 0.5, 'para')
    except BaseException:
        continue
    jmdata_map[key] = {}
    content = a.get_article_sentence(jmap['content'])
    sen = a.splitByPeriod(para)
    jmdata_map[key]['content'] = content
    jmdata_map[key]['para'] = para
    jmdata_map[key]['sen'] = sen
    print '---' + key + ' done' + '----'
    i+=1


Pdfminer Parsed time: 39s
Construct Pdf Tree time: 0s
pdflist length: 182
---http://www.cninfo.com.cn/finalpage/2017-06-20/1203635303.PDF done----
Download pdf time: 7s
download http status code: 200
Pdfminer Parsed time: 36s
Construct Pdf Tree time: 0s
pdflist length: 166
---http://disclosure.szse.cn/finalpage/2017-04-11/1203274908.PDF done----
Download pdf time: 2s
download http status code: 200
Pdfminer Parsed time: 17s
Construct Pdf Tree time: 0s
pdflist length: 46
---http://www.cninfo.com.cn/finalpage/2017-07-26/1203737887.PDF done----
Download pdf time: 6s
download http status code: 200
Pdfminer Parsed time: 35s
Construct Pdf Tree time: 0s
pdflist length: 86
---http://disclosure.szse.cn/finalpage/2017-08-25/1203872097.PDF done----
Download pdf time: 1s
download http status code: 200
Pdfminer Parsed time: 6s
Construct Pdf Tree time: 0s
pdflist length: 11
---http://disclosure.szse.cn/finalpage/2016-10-18/1202765393.PDF done----
Download pdf time: 1s
download http status code: 200
P

In [262]:
def test(a):
    x = list(a)
    x.remove(2)
    return x
a = [1,2,3,4]
print test(a)
print a

[1, 3, 4]
[1, 2, 3, 4]
